# Práctica 1

## Introducción

La [Secretaría de Salud de México](https://www.gob.mx/salud) publica [reportes epidemiológicos semanales](https://www.gob.mx/salud/acciones-y-programas/historico-boletin-epidemiologico) de entre los que se eligió el periodo comprendido entre 2005 y 2015 para agrupar las enfermedades mediante un algoritmo no supervisado llamado $k$-medias. Como referencia, se tiene que la \textbf{Organización Mundial de la Salud} (OMS) realiza un agrupamiento de enfermedades en un instrumento estadístico llamado \textbf{Clasificación Estadística Internacional de Enfermedades y Problemas Relacionados con la Salud} (CIE) \cite{cie}.

## Preguntas de investigación

* ¿Es posible agrupar estos datos mediante *k*-medias de una manera similar a la que se propone por la OMS?
* ¿Qué características son adecuadas para clasificar estos datos?
* ¿Qué tienen en común los agrupamientos obtenidos a partir de *k*-medias?

## Objetivos

Agrupar enfermedades mediante un algoritmo de $k$-medias a partir de los datos de series de tiempo con que son reportadas en los boletines epidemiológicos de la Secretaría de Salud de México.

## Caracterización de datos

La Secretaría de Salud de México publica boletines epidemiológicos en los que se detalla semanalmente el número de casos registrados por enfermedad en cada estado de la república mexicana. Estos boletines pueden descargarse desde la [página de la Secretaría de Salud](https://www.gob.mx/salud/acciones-y-programas/direccion-general-de-epidemiologia-boletin-epidemiologico). Estos archivos se descargaron por medio de un programa que accede de manera iterativa a las direcciones URL de los archivos. En general, las direcciones de los archivos pudieron predecirse a partir de la observación de patrones en algunas direcciones por año. Por ejemplo, el boletín de la semana epidemiológica 25 de 2008 está disponible en https://www.gob.mx/cms/uploads/attachment/file/13605/2008_sem25.pdf y el del boletín 51 del mismo año en https://www.gob.mx/cms/uploads/attachment/file/13631/2008_sem51.pdf, de donde se puede observar que sólo cambian los dígitos que siguen a `sem` y preceden a `.pdf`. Esto no sucede en el año 2013, cuyas direcciones no parecen seguir ningún patrón rápidamente predecible, por lo que se optó por descargar manualmente los boletines de ese año. Los boletines se encuentran en formato [PDF](https://acrobat.adobe.com/mx/es/acrobat/pdf-reader.html), salvo los del año 2011, cuyas páginas por separado se hallan en formato PDF comprimidas en formato ZIP. Con el fin de tener los boletines en el mismo tipo de archivo y formato, las páginas de los boletines de 2011 fueron extraídas y agrupadas en un solo archivo PDF por semana epidemiológica mediante el uso de la herramienta [PDF Mergy](https://pdfmerge.w69b.com/).

Cada boletín corresponde a una semana epidemiológica del año, de modo que generalmente se tienen 52 boletines por año, excepto en 2008 y 2014 en que se cuenta con 53. La información semanal de casos registrados para cada enfermedad y estado de la República se muestra por página en forma de cuadro. 

En los cuadros de casos registrados, las filas contienen los estados de la república mexicana con una fila al final que representa el total, y en las columnas se tienen las enfermedades. Cada página que contiene estos cuadros incluye todos los estados de la República y de una a cuatro columnas de enfermedad. Cada columna puede estar subdividida en uno, dos o tres años; cada año contiene una subdivisión que puede incluir la cantidad de casos registrados en la semana epidemiológica del boletín, el acumulado del año a la fecha de la semana epidemiológica del boletín o la cantidad de hombres y mujeres que reportaron la afección en la semana epidemiológica o a lo largo del año, esto último cuando se añade un encabezado para indicarlo. Además, las celdas de las enfermedades incluyen el nombre, la edición de revisión de la [CIE](https://es.wikipedia.org/wiki/CIE-10) y la CIE asignada por la Secretaría de Salud de México. Los casos se registran con números arábigos, pero puede aparecer una siglas cuya nomenclatura se especifica en los boletines como sigue:


* -: No se presentaron casos en la semana epidemiológica.
* n. d.: Información no disponible.
* n. e.: Información no enviada por la entidad federativa.
* n. a.: No aplica.
* s. n.: Sin notificación por la entidad federativa.

Dicha nomenclatura suele variar en ausencia espacios entre palabras o de uno o los dos puntos de manera difícil de predecir e incluso observar a simple vista. Asimismo, cuando los números de casos exceden los cientos, pueden encontrarse separaciones en grupos de tres dígitos por comas o espacios (uno o más), o carecer de ellas. Cabe destacar que las tipografías varían a lo largo de los años y se presentan boletines que utilizan comillas de manera irregular para enmarcar datos. 

Para extraer los datos de los archivos PDF se diseñó un algoritmo que utiliza la herramienta [tabula-py](https://github.com/chezou/tabula-py) y [PyPDF2](https://pypi.org/project/PyPDF2/) que se ejecutan en lenguaje [Python](https://www.python.org/). Con la primera se extrae el contenido de archivos \texttt{PDF} mediante especificación de algunos parámetros, y la segunda permite leer archivos PDF y, entre otras funciones, extraer el número de páginas.

En el algoritmo se elige como punto de partida para la lectura de datos la aparición de las cadenas de texto **CUADRO 3** y **MENINGITIS** puesto que todos los documentos \texttt{PDF} presentaban esta información en el primer cuadro de interés. Sin embargo y pese a esta extracción, los datos volcados en archivos de formato CSV contenían numerosos errores de lectura, por ello se pasaron por otro archivo escrito en Python que limpia de cada registro los números de casos reportados, el estado de la república mexicana, los nombres de enfermedad y las CIEs asignadas. Tras limpiar cada archivo generado por el programa que extraía la información, se buscaron errores mediante expresiones regulares, técnica de búsqueda de caracteres o estructuras de caracteres definida por [Ken Thompson](http://doi.acm.org/10.1145/363347.363387). Esta búsqueda evidenció errores de formato en las CIEs del grupo T63, además de errores en el número de casos reportados en miles de registros, mismos que fueron corregidos manualmente.

Como resultado de este proceso, se extrajeron 784660 registros con 169 CIEs distintas que incluyen el año reportado, la semana epidemiológica (SE) correspondiente, el estado de la república mexicana donde se informó de los incidentes, el número de casos registrados, el nombre de la enfermedad y la CIE asignada por la OMS. Un ejemplo de estos datos es

Año | SE | Estado | Enfermedad | Casos | CIE
--|--|--|--|--|--
2006 | 12 | Querétaro | Shigelosis | 0 | A03
2010 | 8 | Zacatecas | Conjuntivitis | 165 | B30
2014 | 29 | Durango | Asma | 85 | J45

Debido a que estos datos representan una extensa colección de información asociada al paso del tiempo, se ha decidido tratarlos en conjuntos como series de tiempo con el fin de procesarlos y reducir sus características para, ulteriormente, utilizarlos como datos de entrada en un algoritmo de $k$-medias. Por lo tanto, en primer lugar se separaron los datos por CIE. A continuación, se normalizaron los casos registrados entre el [número de derechohabientes del año correspondiente](http://www3.inegi.org.mx/sistemas/sisept/default.aspx?t=msoc01) con el fin de tener una medición comparable de enfermedades. Luego, para cada CIE, se obtuvo en R la tendencia de la serie de tiempo con [`lm`](https://stat.ethz.ch/R-manual/R-devel/library/stats/html/lm.html). Posteriormente, se restó la tendencia a la serie de tiempo usando [`detrend`](https://www.rdocumentation.org/packages/pracma/versions/1.9.9/topics/detrend). Para una mejor visualización de estos datos, se comparte el ejemplo de los datos de la fiebre tifoidea con el número de casos reportados por derechohabientes (puntos), la regresión lineal (línea negra) y los datos menos tendencia (línea roja)

![](../img/tendencia.png)

Con esto, se convierte la serie de tiempo en una señal susceptible de ser tratada por las funciones [`periodogram`](https://www.rdocumentation.org/packages/TSA/versions/1.2/topics/periodogram) y [`acf`](https://www.rdocumentation.org/packages/forecast/versions/8.4/topics/Acf), las cuales, respectivamente, extraen los periodogramas de Fourier y las autocorrelaciones para un retraso de 1 a 52 semanas por CIE, mismo que puede graficarse como

![](../img/fourier.png) ![](../img/autocorr.png)

Con esta reducción de datos a series de tiempo por CIE, se disminuyeron los registros de 784660 a tan solo 169, equivalentes a la cantidad de CIEs registradas. A este conjunto de datos se le asignaron varias combinaciones de características que incluían las frecuencias de Fourier, todas las autocorrelaciones con y sin la pendiente u ordenada en el origen de la tendencia. Tras varias pruebas, se obtuvieron los mejores resultados utilizando

* la pendiente de la tendencia,
* la ordenada en el origen de la tendencia,
* las seis más altas frecuencias de Fourier, y 
* las autocorrelaciones 1, 2, 4, 8, 16 y 32.